In [1]:
# Install packages
!pip install -r requirements.txt >/dev/null
!pip install eep153_tools >/dev/null
!pip install gnupg 
!pip install gspread_pandas >/dev/null

# Import libraries
import numpy as np
import pandas as pd
import warnings
from  scipy.optimize import linprog as lp
import plotly.express as px
from eep153_tools.sheets import read_sheets
import fooddatacentral as fdc


# API Key
apikey = "Y7y8evhwYYL589qVEZzdHBCyKGeBkNdDWQfnWyIV"

## [A] Description of Population of Interest

Our group is interested in the comparison of the minimum cost diets of the inhabitants of **San-Francisco** and **Bakersfield**, two cities chosen based on their differences in cost-of-living, population density, and locale classification (urban vs agricultural, respectively).

## [A] Dietary Reference Intakes

A function that takes as arguments the characteristics of a given person (age, sex) and returns a Pandas.series of Dietary Reference Intakes (DRIs) or "Recommended Daily Allowances" (RDAs) of a variety of nutrients appropriate for the population of interest

In [2]:
# Read diet minimum data
diet_min = pd.read_csv("Spreadsheets/diet_min.csv",index_col=0)
diet_min

,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,,
Energy,---,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,RDA,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",---,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",RDA,150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",RDA,700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",RDA,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",RDA,7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",RDA,80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,RDA,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


In [3]:
# Read diet maximum data
diet_max = pd.read_csv("Spreadsheets/diet_max.csv",index_col=0)
diet_max

,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,,
"Sodium, Na",UL,1500,1900,1900,2200,2200,2300,2300,2300,2300,2300,2300,2300,2300
Energy,NaN,2500,2500,2500,2800,3000,3100,3100,3100,3100,3100,3100,3100,3100


In [4]:
def dietary_ref_intake(age,sex,df):
    """Takes in age and sex, and returns the dietary reference intake for the chosen population"""

    if age <= 3:
        col = 'C 1-3'
    age_ranges = [(4,8),(9,13),(14,18),(19,30),(31,50),(50,100)]
    for age_range in age_ranges:
        if age >= age_range[0] and age <= age_range[1]:
            col = sex + ' ' + str(age_range[0]) + '-' + str(age_range[1])
    return pd.Series(df[col])  

In [5]:
# Example of minimum dietary requirements for a male aged 22
dietary_ref_intake(age=22,sex='M',df=diet_min)

Nutrition
Energy                            2400.0
Protein                             56.0
Fiber, total dietary                33.6
Folate, DFE                        400.0
Calcium, Ca                       1000.0
Carbohydrate, by difference        130.0
Iron, Fe                             8.0
Magnesium, Mg                      400.0
Niacin                              16.0
Phosphorus, P                      700.0
Potassium, K                      4700.0
Riboflavin                           1.3
Thiamin                              1.2
Vitamin A, RAE                     900.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      90.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)          120.0
Zinc, Zn                            11.0
Name: M 19-30, dtype: float64

In [6]:
# Example of maximum dietary requirements for a male aged 22
dietary_ref_intake(age=22,sex='M',df=diet_max)

Nutrition
Sodium, Na    2300
Energy        3100
Name: M 19-30, dtype: int64

## [A] Data on prices for different foods

We constructed a google spreadsheet of the prices of approximately 28 ~ 29 different food products chosen from 2 different methods:  

* **US Consumer-Expenditure Survey Top-Results**: The first bin of foods were chosen as the most popular products per each main food groups (i.e. vegetables, fruits, meats, dairies) as concluded from the US Consumer-Expenditure Survey. This represents the baseline foods we wanted to establish to compare the two cities within our project.  Here is a link to what we looked at: https://www.ers.usda.gov/data-products/ag-and-food-statistics-charting-the-essentials/food-availability-and-consumption/  
* **Insta-Cart 2021 Most Popular Groceries by City**: We then compiled the most popular groceries per region by searching by city through Instacart's 2021 Delivered project: https://www.instacart.com/2021-delivered/

### Identifying and Uploading FDC

In [7]:
df = pd.read_csv('Spreadsheets/grocery_names.csv')
df['Food'] = df['Item']
df

,Item,Food
0,Hass Avocado,Hass Avocado
1,Tofu,Tofu
2,Lacinato Kale,Lacinato Kale
3,Oat Milk,Oat Milk
4,Flat Leaf Parsley,Flat Leaf Parsley
5,Maradol Papayas,Maradol Papayas
6,Gemelli Pasta,Gemelli Pasta
7,Roma Tomatoes,Roma Tomatoes
8,Serrano Peppers,Serrano Peppers
9,Cilantro,Cilantro


In [8]:
# (IGNORE FOR PROJECT) idea for further automation/ease here: if you inputted a dataframe with your 
# chosen foods having already designated for the "foodCategory" column, that way you could have some 
# kind of if/else statement that filters to find the FIRST index row with that DESIRED (and more accurate) 
# food category as the value in that column of the FDC dataframe. This way you could weed out rando ones, 
# like when we get "Avocado Chunks" as the SECOND result in a search for Hass Avocadoes. 

import fooddatacentral as fdc
import warnings

apikey = 'Y7y8evhwYYL589qVEZzdHBCyKGeBkNdDWQfnWyIV'

def find_FDC_index(API_key, food_list, num_rows): 
    
    fdc_index_results = pd.DataFrame(columns=['Food', 'fdcId', 'description', 'foodCategory'])
    
    for food in food_list: 
        df_food = fdc.search(apikey, food)
        df_food_reduced = df_food[['fdcId', 'description', 'foodCategory']]
        df_food_reduced['Food'] = str(food)
        df_food_4_results = df_food_reduced.iloc[:num_rows]
        fdc_index_results = fdc_index_results.append(df_food_4_results, ignore_index = True)
        
    return fdc_index_results 

In [9]:
get_fdcIds = find_FDC_index(apikey, df['Item'], 1)
get_fdcIds

/tmp/ipykernel_462/3561763109.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_food_reduced['Food'] = str(food)


,Food,fdcId,description,foodCategory
0,Hass Avocado,2609822,"HASS AVOCADOS, HASS",Pre-Packaged Fruit & Vegetables
1,Tofu,2294522,TOFU,Other Meats
2,Lacinato Kale,2495872,"LACINATO TUSCAN KALE CUT SUPER GREENS, LACINAT...",Pre-Packaged Fruit & Vegetables
3,Oat Milk,2257046,"Oat milk, unsweetened, plain, refrigerated",Beverages
4,Flat Leaf Parsley,170416,"Parsley, fresh",Vegetables and Vegetable Products
5,Maradol Papayas,169926,"Papayas, raw",Fruits and Fruit Juices
6,Gemelli Pasta,2618997,"GEMELLI & SPINACH PASTA SALAD, GEMELLI & SPINACH",Deli Salads
7,Roma Tomatoes,1608947,ROMA TOMATOES,Pre-Packaged Fruit & Vegetables
8,Serrano Peppers,2390015,SERRANO PEPPERS,"Pickles, Olives, Peppers & Relishes"
9,Cilantro,399147,CILANTRO,Herbs & Spices


In [10]:
fdcId_array = get_fdcIds['fdcId']
print(fdcId_array)

0     2609822
1     2294522
2     2495872
3     2257046
4      170416
5      169926
6     2618997
7     1608947
8     2390015
9      399147
10    1850647
11    2341591
12    2341550
13    2630019
14    1545857
15     578523
16     173945
17     576920
18     170050
19    1909132
20    2679762
21    1635045
22    2431121
23    2671058
24    2090362
25    1942314
26    1663044
27    2345315
Name: fdcId, dtype: object


In [16]:
file_path = "Spreadsheets/fdc_ids.csv"

# Save the DataFrame to a CSV file
fdcId_array.to_csv(file_path, index=False)

### Notes from FDC ID DataFrame: 
- for Gemelli Pasta, pd.iloc[0] will get fdcId = 2618997, change to fdcId = 1124597
- "pressed Juices" are fucked up, need to do that search separately
- "Bread" is outputting some cookies and biscuits shit, replace with fdcId = 1913550

In [17]:
gemelli_fdcID = 1124597
bread_fdcID = 1913550
pressedjuice_fdcID = 2095092

In [18]:
# getting FDC Codes

import fooddatacentral as fdc
import warnings
df1 = fdc.search(apikey, 'Cold Pressed Juice')
df1.head(5)

,fdcId,description,dataType,gtinUpc,publishedDate,brandOwner,brandName,ingredients,marketCountry,foodCategory,...,foodMeasures,foodAttributes,foodAttributeTypes,foodVersionIds,householdServingFullText,shortDescription,commonNames,additionalDescriptions,ndbNumber,subbrandName
0,2075597,COLD PRESSED JUICE,Branded,852043003810,2021-10-28,"The Hain Celestial Group, Inc.",BLUEPRINT,"JUICE FROM: ORGANIC ROMAINE, ORGANIC APPLE, OR...",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,522238,COLD PRESSED JUICE,Branded,857134006749,2019-04-01,WHOLE FOODS MARKET,NaN,"ORANGE, TURMERIC, CURRY POWDER, BALCK PEPPER",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],2 OZA,NaN,NaN,NaN,NaN,NaN
2,1919466,COLD PRESSED JUICE,Branded,653341767899,2021-07-29,"A.M.S. Manufacturing, Inc.",JONI JUICE,"APPLE, KALE, MINT, LEMON, GINGER, SPIRULINA, M...",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
3,2095092,COLD PRESSED JUICE,Branded,813377021864,2021-10-28,Urban Remedy,365 WHOLE FOODS MARKET,"PINEAPPLE JUICE, LIME JUICE, GINGER JUICE.",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
4,1889790,COLD PRESSED JUICE,Branded,854649005960,2021-07-29,Pressery LLC,WHOLE FOODS MARKET,"CARROTS, BEETS, LEMON, GINGER.",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN


### DataFrames from Google Sheets

In [92]:
def read_sheet(city):
    df = pd.read_csv("Spreadsheets/" + city + ".csv")
    df = df.iloc[:,:5].dropna(subset=['FDC'])
    df = df.reset_index(drop=True)
    df['FDC'] = df['FDC'].astype(int)
    return df
    return df

In [93]:
sf = read_sheet("San_Francisco")
bakersfield = read_sheet("Bakersfield")

sf

,Item,Quantity,Units,Price,FDC
0,Hass Avocado,6.0,oz,2.00,2609822
1,Tofu,14.0,oz,3.49,2294522
2,Lacinato Kale,1.0,lb,3.49,2495872
3,Oat Milk,64.0,oz,5.99,2257046
4,Flat Leaf Parsley,2.0,oz,1.79,170416
5,Maradol Papayas,1.0,lb,3.99,169926
6,Gemelli Pasta,7.0,oz,2.79,1124597
7,Roma Tomatoes,1.0,lb,3.49,1608947
8,Serrano Peppers,0.1,lb,0.75,2390015
9,Cilantro,2.8,oz,1.69,399147


### Unit Conversion 

In [94]:
def convert(df):
    # Convert food quantities to FDC units
    df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

    # Now may want to filter df by time or place--need to get a unique set of food names.
    df['FDC Price'] = df['Price']/df['FDC Quantity']

    df.dropna(how='any') # Drop food with any missing data

    # To use minimum price observed
    Prices = df.groupby('Item',sort=False)['FDC Price'].min()

    return Prices

In [95]:
sf_price = convert(sf)
bakersfield_price = convert(bakersfield)

DimensionalityError: Cannot convert from 'femtoliter * ounce' ([length] ** 3 * [mass]) to 'deciliter' ([length] ** 3)

# [A] Nutritional content of different foods

Now, using the FDC ID, we look up the nutritional content of the grocery lists we've created to construct DataFrames containing the results

In [97]:
def content(city):
    df = read_sheet(city)
    D = {}
    count = 0
    for item in  df.Item.tolist():
        try:
            FDC = df.loc[df.Item==item,:].FDC[count]
            count+=1
            D[item] = fdc.nutrients(apikey,FDC).Quantity
        except AttributeError: 
            warnings.warn("Couldn't find FDC Code %s for item %s." % (item,FDC))
    return pd.DataFrame(D,dtype=float)

In [98]:
sf_content = content(sf)
bakersfield_content = content(bakersfield)

sf_content

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U13'), dtype('float64')) -> None

In [99]:
sf_content

NameError: name 'sf_content' is not defined

## [A] Solution

Here we create a single function `solve_subsistence_problem`.  By
isolating the logic of constructing and solving the subsistence
problem into a stand-alone function we reduce the scope for bugs, and
this modular approach at the same time makes testing easier.

We take the different pieces of the puzzle we&rsquo;ve developed and
put them together in the form of a linear program we can solve.
Recall that the mathematical problem we&rsquo;re trying to solve is
$$
    \min_x p'x
$$
such that
$$
     Ax \geq b
$$
If we buy a bag of groceries with quantities given by $x$, the total
cost of the bag of groceries is the inner product of prices and
quantities.  Since we&rsquo;ve converted our units above, this gives us a
vector of prices where quantities are all in 100 g or ml units.

The following code block defines a function
`solve_subsistence_problem`, which takes as arguments a dataframe
mapping different foods to nutrients; a series of prices for those
same foods; a series giving dietary recommended intake (DRI) minimums;
and a series giving dietary recommended maximums.



In [100]:
def solve_subsistence_problem(FoodNutrients,Prices,dietmin,dietmax,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    try: 
        p = Prices.apply(lambda x:x.magnitude)
    except AttributeError:  # Maybe not passing in prices with units?
        warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
        p = Prices

    p = p.dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(dietmin.index)]
    Amin = Amin.reindex(dietmin.index,axis=0)
    idx = Amin.index.to_frame()
    idx['type'] = 'min'
    #Amin.index = pd.MultiIndex.from_frame(idx)
    #dietmin.index = Amin.index
    
    Amax = Aall.loc[Aall.index.intersection(dietmax.index)]
    Amax = Amax.reindex(dietmax.index,axis=0)
    idx = Amax.index.to_frame()
    idx['type'] = 'max'
    #Amax.index = pd.MultiIndex.from_frame(idx)
    #dietmax.index = Amax.index

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([dietmin,
                   -dietmax]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
        
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

In [101]:
def solution(city,group):
    Prices, FoodNutrients = match(city)
    tol = 1e-6

    result = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

    print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

    # Put back into nice series
    diet = result.diet

    print("\nDiet (in 100s of grams or milliliters):")
    print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
    print()

    tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
    print("\nWith the following nutritional outcomes of interest:")
    print(tab)
    print()

    print("\nConstraining nutrients are:")
    excess = tab.diff(axis=1).iloc[:,1]
    print(excess.loc[np.abs(excess) < tol*100].index.tolist())


In [102]:
def match(city):
    if city == 'San_Francisco':
        return sprouts_price, sprouts_nutrient
    if store == 'Bakersfield':
        return safeway_price, safeway_nutrient

In [103]:
solution('San_Francisco', 'M 14-18')

NameError: name 'sprouts_price' is not defined

In [104]:
solution('Bakersfield', 'M 14-18')

NameError: name 'store' is not defined